In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import os
import glob
from typing import Optional

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
import torchvision
import torchvision.transforms as T

In [2]:
dataset_path = '/home/hari/data/Personal/flower_photos'
classes = glob.glob(os.path.join(dataset_path, '**'))
images = glob.glob(os.path.join(dataset_path, '**/*.jpg'))
print(len(classes))
print(len(images))

5
3670


In [3]:
for c in classes:
    images = glob.glob(os.path.join(dataset_path, c, '*.jpg'))
    print(len(images))

641
699
633
799
898


In [4]:
transforms = T.Compose([
    T.ToTensor(),
    T.Resize((256, 256), antialias=False),
])

In [5]:
class Dataset(data.Dataset):
    def __init__(self, root_directory, transforms=None, train=True):
        self.root_directory = root_directory
        self.transforms = transforms
        self.train_size = 0.8
        self.train = train
        self.images = self.__train_validation_split__()
        classes = [c.split('/')[-1] for c in sorted(glob.glob(os.path.join(root_directory, '**')))]
        self.class_dict = {c : i for i, c in enumerate(classes)}

    def __train_validation_split__(self):
        images = glob.glob(os.path.join(self.root_directory, '**/*.jpg'))
        _size = int(len(images) * self.train_size) if self.train else int(len(images) * (1 - self.train_size))
        indices = np.random.choice(len(images), _size, replace=False)
        images = np.array(images)[indices]
        return images

    def __class_tonumeric__(self, label):
        return self.class_dict[label]
    
    def __getitem__(self, index):
        path = self.images[index]
        label = path.split('/')[-2]
        label = self.__class_tonumeric__(label)
        image = cv.imread(path)
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        if self.transforms is not None:
            image = self.transforms(image)
        return image, label
    
    def __len__(self):
        return len(self.images)

In [6]:
dataset = Dataset(root_directory=dataset_path, transforms=transforms, train=True)
validset = Dataset(root_directory=dataset_path, transforms=transforms, train=False)
dataloader = data.DataLoader(dataset, batch_size=4, shuffle=True, drop_last=True)
validloader = data.DataLoader(validset, batch_size=1, shuffle=True, drop_last=True)
X, y = next(iter(dataloader))
print(X.shape, y.shape)

torch.Size([4, 3, 256, 256]) torch.Size([4])


In [7]:
class LearnedReLU(nn.Module):
    def __init__(self, learning_epochs):
        super().__init__()
        self.learning_epochs = learning_epochs
        positive_coefficient = torch.normal(torch.ones(1), torch.ones(1))
        negative_coefficient = torch.normal(torch.zeros(1), torch.ones(1))
        self.positive_coefficient = nn.Parameter(positive_coefficient)
        self.negative_coefficient = nn.Parameter(negative_coefficient)

    def forward(self, X, epoch=None):
        _p_copy = self.positive_coefficient.data.clone()
        _n_copy = self.negative_coefficient.data.clone()
        if epoch is None or epoch >= self.learning_epochs:
            _p_copy, _n_copy = nn.Parameter(torch.ones(1)), nn.Parameter(torch.zeros(1))
        out = torch.where(X>=0, _p_copy * X, _n_copy * X)
        return out

In [8]:
class PerturbedReLU(nn.Module):
    def __init__(self, epsilon=0.01):
        super().__init__()
        self.epsilon = epsilon
    
    def forward(self, X):
        activated = F.relu(X)
        perturbation = torch.normal(torch.ones(X.shape)*self.epsilon, torch.ones(X.shape)*self.epsilon)
        return activated + perturbation

In [9]:
class ControlNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(11, 11), stride=(5, 5))
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(9, 9), stride=(3, 3))
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(7, 7), stride=(1, 1))
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=(5, 5), stride=(1, 1))
        self.linear1 = nn.Linear(1024, 512)
        self.linear2 = nn.Linear(512, 5)
        self.flatten = nn.Flatten()
        self.act = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, X):
        out = self.conv1(X)
        out = self.act(out)
        out = self.conv2(out)
        out = self.act(out)
        out = self.conv3(out)
        out = self.act(out)
        out = self.conv4(out)
        out = self.act(out)
        out = self.flatten(out)
        out = self.linear1(out)
        out = self.act(out)
        out = self.linear2(out)
        out = self.softmax(out)
        return out

In [10]:
# class TestNetwork(nn.Module):
#     def __init__(self, activation_learning_epochs):
#         super().__init__()
#         self.activation_learning_epochs = activation_learning_epochs
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(11, 11), stride=(5, 5))
#         self.act1 = LearnedReLU(activation_learning_epochs)
#         self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(9, 9), stride=(3, 3))
#         self.act2 = LearnedReLU(activation_learning_epochs)
#         self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(7, 7), stride=(1, 1))
#         self.act3 = LearnedReLU(activation_learning_epochs)
#         self.conv4 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=(5, 5), stride=(1, 1))
#         self.act4 = LearnedReLU(activation_learning_epochs)
#         self.linear1 = nn.Linear(1024, 512)
#         self.act5 = LearnedReLU(activation_learning_epochs)
#         self.linear2 = nn.Linear(512, 5)
#         self.flatten = nn.Flatten()
#         self.softmax = nn.Softmax(dim=1)

#     def forward(self, X, epoch):
#         out = self.act1(self.conv1(X), epoch)
#         out = self.act2(self.conv2(out), epoch)
#         out = self.act3(self.conv3(out), epoch)
#         out = self.act4(self.conv4(out), epoch)
#         out = self.flatten(out)
#         out = self.act5(self.linear1(out), epoch)
#         out = self.softmax(self.linear2(out))
#         return out

class TestNetwork1(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(11, 11), stride=(5, 5))
        self.act1 = PerturbedReLU(epsilon=0.01)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(9, 9), stride=(3, 3))
        self.act2 = PerturbedReLU(epsilon=0.01)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(7, 7), stride=(1, 1))
        self.act3 = PerturbedReLU(epsilon=0.01)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=(5, 5), stride=(1, 1))
        self.act4 = PerturbedReLU(epsilon=0.01)
        self.linear1 = nn.Linear(1024, 512)
        self.act5 = PerturbedReLU(epsilon=0.01)
        self.linear2 = nn.Linear(512, 5)
        self.flatten = nn.Flatten()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, X):
        out = self.act1(self.conv1(X))
        out = self.act2(self.conv2(out))
        out = self.act3(self.conv3(out))
        out = self.act4(self.conv4(out))
        out = self.flatten(out)
        out = self.act5(self.linear1(out))
        out = self.softmax(self.linear2(out))
        return out

class TestNetwork2(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(11, 11), stride=(5, 5))
        self.act1 = PerturbedReLU(epsilon=0.01)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(9, 9), stride=(3, 3))
        self.act2 = PerturbedReLU(epsilon=0.005)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(7, 7), stride=(1, 1))
        self.act3 = PerturbedReLU(epsilon=0.001)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=(5, 5), stride=(1, 1))
        self.act4 = PerturbedReLU(epsilon=0.0005)
        self.linear1 = nn.Linear(1024, 512)
        self.act5 = PerturbedReLU(epsilon=0)
        self.linear2 = nn.Linear(512, 5)
        self.flatten = nn.Flatten()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, X):
        out = self.act1(self.conv1(X))
        out = self.act2(self.conv2(out))
        out = self.act3(self.conv3(out))
        out = self.act4(self.conv4(out))
        out = self.flatten(out)
        out = self.act5(self.linear1(out))
        out = self.softmax(self.linear2(out))
        return out

In [11]:
test_net_1 = TestNetwork1()
test_net_2 = TestNetwork2()
ctrl_net = ControlNetwork()
loss = nn.CrossEntropyLoss()
test_optimizer_1 = optim.Adam(test_net_1.parameters(), lr=1e-4)
test_optimizer_2 = optim.Adam(test_net_2.parameters(), lr=1e-4)
ctrl_optimizer = optim.Adam(ctrl_net.parameters(), lr=1e-4)

In [12]:
def get_accuracy(validloader, ctrl_net, test_net1, test_net2=None):
    ctrl_acc, test_acc1, test_acc2 = 0.0, 0.0, 0.0
    for X, y in validloader:
        with torch.no_grad():
            ctrl_out = ctrl_net(X)
            test_out1 = test_net1(X)
            if test_net2 is not None:
                test_out2 = test_net2(X)
        ctrl_acc += (ctrl_out.argmax(dim=1)==y).sum().item()
        test_acc1 += (test_out1.argmax(dim=1)==y).sum().item()
        if test_net2 is not None:
            test_acc2 += (test_out2.argmax(dim=1)==y).sum().item()
            return ctrl_acc/len(validloader), test_acc1/len(validloader), test_acc2/len(validloader)
        return ctrl_acc/len(validloader), test_acc1/len(validloader)

In [13]:
file_path = '/home/hari/Development/github/personal/ActivationFunctions/conventional_perturbed_constant_decaying.txt'
file = open(file_path, 'w')

In [14]:
num_epochs = 30
ctrl_losses, test_losses = [], []
line1 = f'{num_epochs} epochs\n'
line2 = 'Conventional ReLU vs Rerturbed ReLU with constant perturbation vs Perturbed ReLU with decaying perturbation on feature extraction\n'
file.write(line1)
file.write(line2)
for epoch in range(num_epochs):
    ctrl_running_loss, test_running_loss_1, test_running_loss_2 = 0.0, 0.0, 0.0
    for i, (X, y) in enumerate(dataloader):
        if i % 100 == 99:
            ctrl_acc, test_acc_1, test_acc_2 = get_accuracy(validloader, ctrl_net, test_net_1, test_net_2)
        ctrl_optimizer.zero_grad()
        test_optimizer_1.zero_grad()
        test_optimizer_2.zero_grad()
        ctrl_out = ctrl_net(X)
        test_out_1 = test_net_1(X)
        test_out_2 = test_net_2(X)
        ctrl_loss = loss(ctrl_out, y)
        ctrl_loss.backward()
        ctrl_optimizer.step()
        test_loss_1 = loss(test_out_1, y)
        test_loss_1.backward()
        test_optimizer_1.step()
        test_loss_2 = loss(test_out_2, y)
        test_loss_2.backward()
        test_optimizer_2.step()
        ctrl_running_loss += ctrl_loss.item()
        test_running_loss_1 += test_loss_1.item()
        test_running_loss_2 += test_loss_2.item()
        if i % 100 == 99:
            curr_ctrl_loss = ctrl_running_loss / 100
            ctrl_acc, test_acc_1, test_acc_2 = get_accuracy(validloader, ctrl_net, test_net_1, test_net_2)
            line = f'Epoch {epoch + 1} | Batch {i + 1}\nControl Accuracy: {ctrl_acc:.3f} | Test Constant Accuracy: {test_acc_1:.3f} | Test Decaying Accuracy: {test_acc_2:.3f}\nControl Loss: {ctrl_running_loss:.3f} | Test Constant Loss: {test_running_loss_1:.3f} | Test Decaying Loss: {test_running_loss_2:.3f}'
            with open(file_path, 'a') as file:
                file.write(line + '\n')
            ctrl_running_loss = 0.0
            test_running_loss_1 = 0.0
            test_running_loss_2 = 0.0
            print(line)
    print('----------------------------------------------------------------------------------')

# file.close()

Epoch 1, Batch 100, Control Accuracy: 0.256, Test Constant Accuracy: 0.274, Test Decaying Accuracy: 0.332, Control Loss: 160.429, Test Constant Loss: 160.898, Test Decaying Loss: 159.848
Epoch 1, Batch 200, Control Accuracy: 0.288, Test Constant Accuracy: 0.274, Test Decaying Accuracy: 0.340, Control Loss: 158.071, Test Constant Loss: 159.564, Test Decaying Loss: 157.325
Epoch 1, Batch 300, Control Accuracy: 0.239, Test Constant Accuracy: 0.274, Test Decaying Accuracy: 0.303, Control Loss: 158.940, Test Constant Loss: 161.284, Test Decaying Loss: 154.085
Epoch 1, Batch 400, Control Accuracy: 0.303, Test Constant Accuracy: 0.274, Test Decaying Accuracy: 0.432, Control Loss: 155.291, Test Constant Loss: 160.151, Test Decaying Loss: 155.004
Epoch 1, Batch 500, Control Accuracy: 0.323, Test Constant Accuracy: 0.266, Test Decaying Accuracy: 0.401, Control Loss: 153.224, Test Constant Loss: 158.999, Test Decaying Loss: 151.544
Epoch 1, Batch 600, Control Accuracy: 0.266, Test Constant Accura